# Provides a Summary of Garmin Daily Monitoring Data

In [1]:
from IPython.display import display, Markdown
import snakemd

import fitfile
from garmindb import GarminConnectConfigManager
from garmindb.garmindb import GarminDb, Attributes, Weight, Stress, RestingHeartRate, IntensityHR, Sleep
from garmindb.garmindb import MonitoringDb, Monitoring, MonitoringHeartRate, MonitoringIntensity, MonitoringClimb

from jupyter_funcs import format_number


gc_config = GarminConnectConfigManager()
db_params_dict = gc_config.get_db_params()
garmin_db = GarminDb(db_params_dict)
garmin_mon_db = MonitoringDb(db_params_dict)

measurement_system = Attributes.measurements_type(garmin_db)
unit_strings = fitfile.units.unit_strings[measurement_system]
distance_units = unit_strings[fitfile.units.UnitTypes.distance_long]

doc = snakemd.new_doc()

doc.add_heading("Monitoring Statistics")
doc.add_paragraph("Analysis of all monitoring data in the database.")

def __get_col_stats(table, col, name, ignore_le_zero=False, time_col=False):
    records = table.row_count(garmin_db)
    if time_col:
        maximum = table.get_time_col_max(garmin_db, col)
    else:
        maximum = format_number(table.get_col_max(garmin_db, col))
    if time_col:
        minimum = table.get_time_col_min(garmin_db, col)
    else:
        minimum = format_number(table.get_col_min(garmin_db, col, None, None, ignore_le_zero))
    if time_col:
        average = table.get_time_col_avg(garmin_db, col)
    else:
        average = format_number(table.get_col_avg(garmin_db, col, None, None, ignore_le_zero))
    if time_col:
        latest = table.get_col_latest(garmin_db, col)
    else:
        latest = format_number(table.get_col_latest(garmin_db, col))
    return [name, records, maximum, minimum, average, latest]

doc.add_table(
    ['Name', 'Records', 'Maximum', 'Minimum', 'Average', 'Latest'],
    [
        __get_col_stats(Weight, Weight.weight, 'Weight'),
        __get_col_stats(Stress, Stress.stress, 'Stress', True),
        __get_col_stats(RestingHeartRate, RestingHeartRate.resting_heart_rate, 'RHR', True),
        __get_col_stats(Sleep, Sleep.total_sleep, 'Sleep', True, True),
        __get_col_stats(Sleep, Sleep.rem_sleep, 'REM Sleep', True, True)
    ])

def __get_monitoring_days(year):
    days = Monitoring.get_days(garmin_mon_db, year)
    days_count = len(days)
    if days_count > 0:
        first_day = days[0]
        last_day = days[-1]
        span = last_day - first_day + 1
    else:
        span = 0
    doc.add_paragraph(f"{year} Days with data ({days_count} count vs {span} span): {days}")
    for index in range(days_count - 1):
        day = int(days[index])
        next_day = int(days[index + 1])
        if next_day != day + 1:
            day_str = str(fitfile.conversions.day_of_the_year_to_datetime(year, day))
            next_day_str = str(fitfile.conversions.day_of_the_year_to_datetime(year, next_day))
            doc.add_paragraph(f"Days gap between {day} ({day_str}) and {next_day} ({next_day_str})")
    return days_count

def __get_monitoring_months(year):
    months = Monitoring.get_month_names(garmin_mon_db, year)
    doc.add_paragraph(f"{year} Months with data ({len(months)}): {months}")

doc.add_heading("Monitoring Records Coverage", 3)
doc.add_paragraph("This shows periods that data has been downloaded for. "
                    "Not seeing data for days you know Garmin has data? "
                    "Change the starting day and the number of days in GarminConnectConfig.json and do a full download.")
years = Monitoring.get_years(garmin_mon_db)
doc.add_paragraph(f"Monitoring records: {Monitoring.row_count(garmin_mon_db)}")
doc.add_paragraph(f"Monitoring Years with data ({len(years)}): {years}")
total_days = 0
for year in years:
    doc.add_heading(str(year), 4)
    __get_monitoring_months(year)
    total_days += __get_monitoring_days(year)
doc.add_paragraph(f"Total days with monitoring data: {total_days}")

display(Markdown(str(doc)))

# Monitoring Statistics

Analysis of all monitoring data in the database.

| Name      | Records | Maximum  | Minimum  | Average  | Latest   |
| --------- | ------- | -------- | -------- | -------- | -------- |
| Weight    | 389     | 95.0     | 84.0     | 89.0     | 87.0     |
| Stress    | 1682157 | 100      | 1        | 29.0     | 22       |
| RHR       | 1138    | 139.0    | 43.0     | 54.0     | 54.0     |
| Sleep     | 2188    | 11:49:00 | 01:21:00 | 07:48:22 | 07:01:00 |
| REM Sleep | 2188    | 03:32:00 | 00:05:00 | 01:16:53 | 02:12:00 |

### Monitoring Records Coverage

This shows periods that data has been downloaded for. Not seeing data for days you know Garmin has data? Change the starting day and the number of days in GarminConnectConfig.json and do a full download.

Monitoring records: 461351

Monitoring Years with data (7): [2019, 2020, 2021, 2022, 2023, 2024, 2025]

#### 2019

2019 Months with data (1): ['Dec']

2019 Days with data (1 count vs 1 span): [365]

#### 2020

2020 Months with data (7): ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul']

2020 Days with data (210 count vs 213 span): [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213]

Days gap between 112 (2020-04-21 00:00:00) and 116 (2020-04-25 00:00:00)

#### 2021

2021 Months with data (2): ['Jul', 'Aug']

2021 Days with data (22 count vs 22 span): [208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229]

#### 2022

2022 Months with data (4): ['Sep', 'Oct', 'Nov', 'Dec']

2022 Days with data (92 count vs 106 span): [260, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362, 363, 364, 365]

Days gap between 260 (2022-09-17 00:00:00) and 275 (2022-10-02 00:00:00)

#### 2023

2023 Months with data (12): ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

2023 Days with data (365 count vs 365 span): [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362, 363, 364, 365]

#### 2024

2024 Months with data (12): ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

2024 Days with data (366 count vs 366 span): [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362, 363, 364, 365, 366]

#### 2025

2025 Months with data (12): ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

2025 Days with data (360 count vs 360 span): [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 360]

Total days with monitoring data: 1416